In [2]:
import myfm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from myfm.utils.benchmark_data import MovieLens100kDataManager

In [3]:
dataset=MovieLens100kDataManager()

In [20]:

# print(dataset.load_rating_predefined_split(3))
ratings = dataset.load_rating_all()
users = dataset.load_user_info()
#Merge tập ratings và users được gom nhóm bởi user_id
result =  pd.merge(ratings, users, on="user_id", how="outer")
#Them giới tính vào tập ratings
newResultGender = result.replace({'M':1,'F':0})
newResultGender


,user_id,movie_id,rating,timestamp,age,gender,occupation,zipcode
0,196,242,3,1997-12-04 15:55:49,49,1,writer,55105
1,196,393,4,1997-12-04 16:11:03,49,1,writer,55105
2,196,381,4,1997-12-04 16:08:48,49,1,writer,55105
3,196,251,3,1997-12-04 16:01:14,49,1,writer,55105
4,196,655,5,1997-12-04 16:09:53,49,1,writer,55105
...,...,...,...,...,...,...,...,...
99995,941,919,5,1997-09-23 21:08:07,20,1,student,97229
99996,941,273,3,1997-09-23 21:10:38,20,1,student,97229
99997,941,1,5,1997-09-23 21:12:24,20,1,student,97229
99998,941,294,4,1997-09-23 21:02:12,20,1,student,97229


In [5]:
#lọc ra các nghê nghiệp của user
occupation = users['occupation'].unique()
occupation

array(['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor'],
      dtype=object)

In [15]:
#Chuyển các nghề nghiệp thành dạng số
newOccupation = {occupation[i]: i for i in range(len(occupation))} 
newOccupation

{'technician': 0,
 'other': 1,
 'writer': 2,
 'executive': 3,
 'administrator': 4,
 'student': 5,
 'lawyer': 6,
 'educator': 7,
 'scientist': 8,
 'entertainment': 9,
 'programmer': 10,
 'librarian': 11,
 'homemaker': 12,
 'artist': 13,
 'engineer': 14,
 'marketing': 15,
 'none': 16,
 'healthcare': 17,
 'retired': 18,
 'salesman': 19,
 'doctor': 20}

In [16]:
#Thêm các nghề nghiệp vừa được số hóa vào tập dữ liệu result
newResultJobs  = result.replace(newOccupation)
# newResultJobs[newResultJobs['occupation']==20]  
newResultJobs


,user_id,movie_id,rating,timestamp,age,gender,occupation,zipcode
0,196,242,3,1997-12-04 15:55:49,49,M,2,55105
1,196,393,4,1997-12-04 16:11:03,49,M,2,55105
2,196,381,4,1997-12-04 16:08:48,49,M,2,55105
3,196,251,3,1997-12-04 16:01:14,49,M,2,55105
4,196,655,5,1997-12-04 16:09:53,49,M,2,55105
...,...,...,...,...,...,...,...,...
99995,941,919,5,1997-09-23 21:08:07,20,M,5,97229
99996,941,273,3,1997-09-23 21:10:38,20,M,5,97229
99997,941,1,5,1997-09-23 21:12:24,20,M,5,97229
99998,941,294,4,1997-09-23 21:02:12,20,M,5,97229


In [19]:
#tập dữ liệu sau khi thêm và mã hóa cả giới tính và nghê nghiệp
newReplace = {'M':1,'F':0} | newOccupation
newResult = result.replace(newReplace)
newResult

,user_id,movie_id,rating,timestamp,age,gender,occupation,zipcode
0,196,242,3,1997-12-04 15:55:49,49,1,2,55105
1,196,393,4,1997-12-04 16:11:03,49,1,2,55105
2,196,381,4,1997-12-04 16:08:48,49,1,2,55105
3,196,251,3,1997-12-04 16:01:14,49,1,2,55105
4,196,655,5,1997-12-04 16:09:53,49,1,2,55105
...,...,...,...,...,...,...,...,...
99995,941,919,5,1997-09-23 21:08:07,20,1,5,97229
99996,941,273,3,1997-09-23 21:10:38,20,1,5,97229
99997,941,1,5,1997-09-23 21:12:24,20,1,5,97229
99998,941,294,4,1997-09-23 21:02:12,20,1,5,97229
